In [45]:
!pip3 install --upgrade pip
!pip3 install openpyxl
!pip3 install geopy
!pip3 install nbformat --upgrade
!pip3 install ipykernel
!pip3 install --upgrade nbformat
!pip install certifi --upgrade
!pip install dash_bootstrap_components
!pip install jupyter-dash




In [2]:
import pandas as pd
import openpyxl

# Load the Excel file into a DataFrame
df = pd.read_excel("DataExport.xlsx")

In [3]:
df

,Application Number,Application Date,Application Type,Address Number,Road Name,Road Type,Object Status Type,Application Status,File Lead,Brief Description,Object Status Date,Ward #,Ward
0,D07-05-16-0003,2016-06-20,Demolition Control,101,WURTEMBURG,Street,Agreement Signed,Post Approval,Douglas James,The application for demolition control propose...,2016-09-26 15:21,Ward 12,RIDEAU-VANIER
1,D07-05-15-0008,2015-12-14,Demolition Control,163,PARKDALE,Avenue,Agreement Registered - Final Legal Clearance G...,Post Approval,Tracey Scaramozzino,An application for Demolition Control for the ...,2018-09-25 09:49,Ward 15,KITCHISSIPPI
2,D07-05-15-0008,2015-12-14,Demolition Control,167,PARKDALE,Avenue,Agreement Registered - Final Legal Clearance G...,Post Approval,Tracey Scaramozzino,An application for Demolition Control for the ...,2018-09-25 09:49,Ward 15,KITCHISSIPPI
3,D07-16-15-0015,2015-12-30,Plan of Subdivision,1626,OLD PRESCOTT,Road,Application File Pending,File Pending,Cheryl Mcwilliams,The proposed plan of subdivision comprises pha...,2016-03-17 09:48,Ward 20,OSGOODE
4,D07-12-16-0002,2016-01-04,Site Plan Control,774,BRONSON,Avenue,Application File Pending,File Pending,Steve Gauthier,12-storey mixed-use building for university st...,2017-12-31 15:52,Ward 17,CAPITAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2041,D07-12-24-0139,2024-10-30,Site Plan Control,99,BILL LEATHEM,Drive,Initial Review Comments Sent,Active,Krishon Walker,A Site Plan Control application to construct a...,2024-11-18 15:25,Ward 24,BARRHAVEN EAST
2042,D07-12-24-0139,2024-10-30,Site Plan Control,2,LEIKIN,Drive,Initial Review Comments Sent,Active,Krishon Walker,A Site Plan Control application to construct a...,2024-11-18 15:25,Ward 24,BARRHAVEN EAST
2043,D07-12-24-0139,2024-10-30,Site Plan Control,20,LEIKIN,Drive,Initial Review Comments Sent,Active,Krishon Walker,A Site Plan Control application to construct a...,2024-11-18 15:25,Ward 24,BARRHAVEN EAST
2044,D09-04-24-0070,2024-11-13,Ontario Heritage Act,79,GUIGUES,Avenue,Report in Progress,Active,Anne Fitzpatrick,"Application to demolish 79 Guigues, a contibut...",2024-11-20 10:03,Ward 12,RIDEAU-VANIER


In [9]:

import pandas as pd
import openpyxl


from geopy.geocoders import Nominatim
import pandas as pd
import plotly.express as px
import os
import certifi
from geopy.geocoders import Nominatim
prop_limit = 20


df = pd.read_excel("DataExport.xlsx")

os.environ['SSL_CERT_FILE'] = certifi.where()

geolocator = Nominatim(user_agent="geoapi")


# Initialize geolocator
geolocator = Nominatim(user_agent="geoapi")

# Function to geocode an address
def geocode_address(address,  country_code='CA'):
    #update address for each valid param
        
    print(address)
    # Try to geocode the address, return None if it fails

    try:
        location = geolocator.geocode(address)
        if location:
            print(location)
            return location.latitude, location.longitude
        else:
            print('Location not found')
            return None, None
    except Exception as e:
        return None, None

# Create the 'Full Address' column
df['Full Address'] = df['Address Number'].astype(str) + ' ' + df['Road Name'] + ' ' + df['Road Type']

# Restrict geocoding to only 10 requests
df['Latitude'], df['Longitude'] = None, None  # Initialize columns
for i, address in enumerate(df['Full Address']):
    print(address)
    print(f"Geocoding address {i + 1} of {prop_limit}")
    if i >= prop_limit:  
        break
    loc  = geocode_address(address)
    print("loc",loc)
    lat, lon = loc
    df.loc[i, 'Latitude'] = lat
    df.loc[i, 'Longitude'] = lon

# Drop rows with missing coordinates
df = df.dropna(subset=['Latitude', 'Longitude'])

# Create the map
fig = px.scatter_mapbox(
    df, 
    lat='Latitude', 
    lon='Longitude', 
    hover_name='Application Number', 
    hover_data=[
        'Application Type', 
        'Application Status', 
        'File Lead', 
        'Brief Description'
    ],
    color='Application Status', 
    zoom=10, 
    height=600
)

# Update the layout of the map
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})

# Show the map
fig.show()


101 WURTEMBURG Street
Geocoding address 1 of 20
101 WURTEMBURG Street
Wurtemburg Street, Sandy Hill, Rideau-Vanier, Ottawa, Eastern Ontario, Ontario, K1N 8Z7, Canada
loc (45.434625433150124, -75.67603581966213)
163 PARKDALE Avenue
Geocoding address 2 of 20
163 PARKDALE Avenue
163, Parkdale Avenue, Tunney's Pasture, Hintonburg, Kitchissippi, Ottawa, Eastern Ontario, Ontario, K1Y 4V8, Canada
loc (45.4070411, -75.7326098)
167 PARKDALE Avenue
Geocoding address 3 of 20
167 PARKDALE Avenue
167, Parkdale Avenue, Tunney's Pasture, Hintonburg, Kitchissippi, Ottawa, Eastern Ontario, Ontario, K1Y 4V8, Canada
loc (45.40706295, -75.73255427998953)
1626 OLD PRESCOTT Road
Geocoding address 4 of 20
1626 OLD PRESCOTT Road
1626, Old Prescott Road, Osgoode, Ottawa, Eastern Ontario, Ontario, K4P 1A9, Canada
loc (45.2495828, -75.5639541)
774 BRONSON Avenue
Geocoding address 5 of 20
774 BRONSON Avenue
774, Bronson Avenue, The Glebe, Capital, Ottawa, Eastern Ontario, Ontario, K1S 4G2, Canada
loc (45.400403, 

In [10]:
import pandas as pd
import os
import certifi

def load_data(file_path):
    """Load the Excel file and preprocess the 'Full Address' column."""
    df = pd.read_excel(file_path)
    df['Full Address'] = df['Address Number'].astype(str) + ' ' + df['Road Name'] + ' ' + df['Road Type']
    return df


In [11]:
from geopy.geocoders import Nominatim

# Initialize geolocator
os.environ['SSL_CERT_FILE'] = certifi.where()
geolocator = Nominatim(user_agent="geoapi")

def geocode_address(address):
    """Geocode a single address and return latitude and longitude."""
    try:
        location = geolocator.geocode(address)
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except Exception as e:
        return None, None

def geocode_dataframe(df, address_col, limit=20):
    """Geocode a DataFrame's address column, limiting the number of requests."""
    df['Latitude'], df['Longitude'] = None, None  # Initialize columns
    for i, address in enumerate(df[address_col]):
        if i >= limit:  # Limit geocoding requests
            break
        lat, lon = geocode_address(address)
        df.loc[i, 'Latitude'] = lat
        df.loc[i, 'Longitude'] = lon
    return df


In [52]:
df.columns

Index(['Application Number', 'Application Date', 'Application Type',
       'Address Number', 'Road Name', 'Road Type', 'Object Status Type',
       'Application Status', 'File Lead', 'Brief Description',
       'Object Status Date', 'Ward #', 'Ward', 'Full Address', 'Latitude',
       'Longitude', 'Processing Time'],
      dtype='object')

In [55]:
import plotly.express as px

def plot_map(df):
    """Create a styled mapbox scatter plot using Plotly."""
    # Create the map
    fig = px.scatter_mapbox(
        df,
        lat='Latitude',
        lon='Longitude',
        hover_name='Application Number',
        hover_data=[
            'Application Type',
            'Application Status',
            'File Lead',
            'Application Date'
            # 'Brief Description'
        ],
        color='Application Status',
        color_discrete_sequence=px.colors.qualitative.Set2,  # Softer color palette
        size_max=15,  # Limit point size
        zoom=0,  # Set to minimum zoom (max zoom out)
        height=700,  # Slightly larger map for better viewing
        opacity=0.8,  # Transparency for map points
    )

    # Update map style
    fig.update_layout(
        mapbox_style="carto-positron",  # Lighter base map for better readability
        mapbox_zoom=0,  # Default zoom level
        mapbox_center={"lat": df['Latitude'].mean(), "lon": df['Longitude'].mean()},  # Center map dynamically
    )

    # Add custom layout tweaks
    fig.update_layout(
        title={
            'text': "Application Locations with Status Highlighted",
            'y': 0.95,  # Title vertical alignment
            'x': 0.5,  # Center the title horizontally
            'xanchor': 'center',
            'yanchor': 'top',
            'font': {'size': 20},  # Larger font for the title
        },
        legend=dict(
            title="Application Status",
            orientation="h",  # Horizontal legend for better readability
            yanchor="bottom",
            y=0.02,
            xanchor="center",
            x=0.5,
        ),
        margin={"r": 10, "t": 40, "l": 10, "b": 10},  # Add some spacing around the map
    )

    # Customize hover labels for better clarity
    fig.update_traces(
        marker=dict(size=10, opacity=0.9),  # Adjust marker size and transparency
        hovertemplate=(
            "<b>%{hovertext}</b><br>"  # Application Number
            "<i>Type:</i> %{customdata[0]}<br>"  # Application Status
            "<i>Status:</i> %{customdata[1]}<br>"  # Application Status
            "<i>Lead:</i> %{customdata[2]}<br>"  # File Lead
            "<i>Application Date:</i> %{customdata[3]}"  # Brief Description
        ),
    )

    return fig


In [42]:
import plotly.express as px

def display_summary_charts(summary):
    """
    Display summary information using charts.
    """
    # Chart 1: Total Applications
    print(f"Total Applications: {summary['Total Applications']}")

    # Chart 2: Breakdown by Application Status
    fig1 = px.pie(
        summary["Status Summary"],
        names="Application Status",  # Ensure this matches the column name in the DataFrame
        values="Count",  # Use "Count" since it represents actual numbers
        title="Application Status Breakdown",
        color_discrete_sequence=px.colors.qualitative.Set2,
    )
    fig1.show()

    # Chart 3: Breakdown by Application Status
    fig2 = px.bar(
        summary["Type Summary"],
        x="Application Status",
        y="Count",
        title="Applications by Type",
        text="Count",
        color="Application Type",
        color_discrete_sequence=px.colors.qualitative.Pastel,
    )
    fig2.update_traces(textposition="outside")
    fig2.show()

    # Chart 4: Top Wards by Application Volume
    fig3 = px.bar(
        summary["Top Wards"],
        x="Ward",
        y="Count",
        title="Top 5 Wards by Application Volume",
        text="Count",
        color="Ward",
        color_discrete_sequence=px.colors.qualitative.Set1,
    )
    fig3.update_traces(textposition="outside")
    fig3.show()

    # Chart 5: Average Processing Time by Status
    fig4 = px.bar(
        summary["Average Processing Time by Status"],
        x="Application Status",
        y="Average Processing Time (days)",
        title="Average Processing Time by Status",
        text="Average Processing Time (days)",
        color="Application Status",
        color_discrete_sequence=px.colors.qualitative.Bold,
    )
    fig4.update_traces(textposition="outside")
    fig4.show()


In [43]:
import plotly.express as px
import plotly.graph_objects as go

def display_summary_charts(summary):
    """
    Display summary information using charts.
    """
    # Chart 1: Total Applications
    print(f"Total Applications: {summary['Total Applications']}")

    # Chart 2: Breakdown by Application Status
    fig1 = px.pie(
        summary["Status Summary"],
        names="Application Status",
        values="Percentage",
        title="Application Status Breakdown",
        color_discrete_sequence=px.colors.qualitative.Set2,
    )
    fig1.show()

    # Chart 3: Breakdown by Application Type
    fig2 = px.bar(
        summary["Type Summary"],
        x="Application Type",
        y="Count",
        title="Applications by Type",
        text="Count",
        color="Application Type",
        color_discrete_sequence=px.colors.qualitative.Pastel,
    )
    fig2.update_traces(textposition="outside")
    fig2.show()

    # Chart 4: Top Wards by Application Volume
    fig3 = px.bar(
        summary["Top Wards"],
        x="Ward",
        y="Count",
        title="Top 5 Wards by Application Volume",
        text="Count",
        color="Ward",
        color_discrete_sequence=px.colors.qualitative.Set1,
    )
    fig3.update_traces(textposition="outside")
    fig3.show()

    # Chart 5: Average Processing Time by Status
    fig4 = px.bar(
        summary["Average Processing Time by Status"],
        x="Application Status",
        y="Average Processing Time (days)",
        title="Average Processing Time by Status",
        text="Average Processing Time (days)",
        color="Application Status",
        color_discrete_sequence=px.colors.qualitative.Bold,
    )
    fig4.update_traces(textposition="outside")
    fig4.show()


In [57]:
if __name__ == "__main__":
    # Load the data
    # file_path = "DataExport.xlsx"
    # df = load_data(file_path)

    # # Geocode the addresses with a limit of 20
    # df = geocode_dataframe(df, address_col='Full Address', limit=200)

    # # Drop rows with missing coordinates
    # df = df.dropna(subset=['Latitude', 'Longitude'])

    # Plot the map
    fig = plot_map(df)
    fig.show()


In [39]:
df


,Application Number,Application Date,Application Type,Address Number,Road Name,Road Type,Object Status Type,Application Status,File Lead,Brief Description,Object Status Date,Ward #,Ward,Full Address,Latitude,Longitude,Processing Time
0,D07-05-16-0003,2016-06-20,Demolition Control,101,WURTEMBURG,Street,Agreement Signed,Post Approval,Douglas James,The application for demolition control propose...,2016-09-26 15:21,Ward 12,RIDEAU-VANIER,101 WURTEMBURG Street,45.434625,-75.676036,98
1,D07-05-15-0008,2015-12-14,Demolition Control,163,PARKDALE,Avenue,Agreement Registered - Final Legal Clearance G...,Post Approval,Tracey Scaramozzino,An application for Demolition Control for the ...,2018-09-25 09:49,Ward 15,KITCHISSIPPI,163 PARKDALE Avenue,45.407041,-75.73261,1016
2,D07-05-15-0008,2015-12-14,Demolition Control,167,PARKDALE,Avenue,Agreement Registered - Final Legal Clearance G...,Post Approval,Tracey Scaramozzino,An application for Demolition Control for the ...,2018-09-25 09:49,Ward 15,KITCHISSIPPI,167 PARKDALE Avenue,45.407063,-75.732554,1016
3,D07-16-15-0015,2015-12-30,Plan of Subdivision,1626,OLD PRESCOTT,Road,Application File Pending,File Pending,Cheryl Mcwilliams,The proposed plan of subdivision comprises pha...,2016-03-17 09:48,Ward 20,OSGOODE,1626 OLD PRESCOTT Road,45.249583,-75.563954,78
4,D07-12-16-0002,2016-01-04,Site Plan Control,774,BRONSON,Avenue,Application File Pending,File Pending,Steve Gauthier,12-storey mixed-use building for university st...,2017-12-31 15:52,Ward 17,CAPITAL,774 BRONSON Avenue,45.400403,-75.699606,727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,D02-02-11-0036,2011-04-29,Zoning By-law Amendment,370,HUNTMAR,Drive,Application File Pending,File Pending,Sean Moore,The applicant is proposing to rezone the subje...,2011-12-06 08:49,Ward 4,KANATA NORTH,370 HUNTMAR Drive,45.287986,-75.919081,221
194,D07-12-11-0166,2011-08-25,Site Plan Control,288,BOOTH,Street,Application File Pending,File Pending,Simon Deiaco,New 7-storey Mixed Use Building with 54 Reside...,2012-01-23 14:30,Ward 14,SOMERSET,288 BOOTH Street,45.40962,-75.710987,151
195,D02-02-11-0066,2011-07-27,Zoning By-law Amendment,6335,PERTH,Street,Application Approved by OMB,Inactive,Cheryl Mcwilliams,The subdivision and zoning would see the devel...,2014-06-23 09:29,Ward 21,RIDEAU-JOCK,6335 PERTH Street,45.188343,-75.848649,1062
198,D02-02-11-0102,2011-10-21,Zoning By-law Amendment,175,RICHMOND,Road,Comment Period in Progress,Post Approval,Allison Hamlin,The Zoning By-law Amendment and Site Plan Cont...,2020-08-11 09:51,Ward 15,KITCHISSIPPI,175 RICHMOND Road,51.451465,-0.319129,3217


In [58]:
import dash
from dash import dcc, html
import dash_bootstrap_components as dbc
from jupyter_dash import JupyterDash
from dash.dependencies import Output, Input  # Add this line

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = dbc.Container(
    [
        dbc.Row(
            dbc.Col(html.H1("Geocoded Address Map with Filtering", className="text-center mt-4 mb-4"), width=12)
        ),
        dbc.Row(
            dbc.Col(
                dcc.Dropdown(
                    id="application-type-filter",
                    options=[{"label": t, "value": t} for t in df['Application Status'].unique()],
                    multi=True,
                    placeholder="Select Application Status",
                ),
                width=12
            )
        ),
        dbc.Row(
            dbc.Col(dcc.Graph(id="map"), width=12)
        ),
    ],
    fluid=True
)

@app.callback(
    Output("map", "figure"),
    [Input("application-type-filter", "value")]
)
def update_map(selected_types):
    """
    Update the map based on selected application types.
    """
    filtered_df = df
    if selected_types:
        filtered_df = df[df['Application Status'].isin(selected_types)]
    return plot_map(filtered_df)

# Run the Dash app in the notebook
app.run_server(mode="inline")